<a href="https://colab.research.google.com/github/viwaumich/research/blob/main/kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages("tidyverse")
install.packages("cluster")
install.packages("factoextra")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘textshaping’, ‘conflicted’, ‘jsonlite’, ‘ragg’, ‘stringr’


Warning message in install.packages("tidyverse"):
“installation of package ‘textshaping’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘ragg’ had non-zero exit status”
Warning message in install.packages("tidyverse"):
“installation of package ‘tidyverse’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘lazyeval’, ‘later’, ‘carData’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘htmlwidgets’, ‘crosstalk’, ‘promises’, ‘estimability’, ‘numDeriv’, ‘mvtnorm’, ‘corrplot’, ‘viridis’, ‘car’, ‘DT’, ‘ellipse’, ‘emmeans’, ‘flashClust’, ‘leaps’, ‘mult

In [2]:
library(tidyverse)  # data manipulation
library(cluster)    # clustering algorithms
library(factoextra) # clustering algorithms & visualization

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa



In [18]:
gag <- read.csv("Gag.csv") #read file
psgl1 <- read.csv("PSGL1.csv") #read file

In [19]:
gag <- na.omit(gag) #remove nas
psgl1 <- na.omit(psgl1) # remove nas

In [20]:
head(gag) #view first 6
head(psgl1) # view first 6

,x,y,z
,<dbl>,<dbl>,<dbl>
1,-13517.3,-1719.89,2174.15
2,-13517.3,-1719.94,2174.51
3,-13517.3,-1719.93,2174.86
4,-13517.6,-1719.37,2176.31
5,-13517.6,-1719.38,2176.71
6,-13516.2,-1718.34,2176.79


,x,y,z
,<dbl>,<dbl>,<dbl>
1,-13529.1,-1762.97,2167.90
2,-13529.4,-1762.49,2168.16
3,-13529.3,-1762.52,2168.71
4,-13529.1,-1762.91,2169.36
5,-13529.3,-1762.46,2169.25
6,-13529.1,-1762.88,2169.70


In [23]:
head(tibble(gag$y)) # making a tibble of a dataframe column variable contents

gag$y
<dbl>
-1719.89
-1719.94
-1719.93
-1719.37
-1719.38
-1718.34


In [44]:
sum = 0 #initialize sum variable
distance <- function(df1, df2){ # function that takes in two dataframes
  for (i in 1:3){ # for i in 1:3 because there are three variables, x,y,z
    sum = sum + (df1[i]-df2[i])^2 # sums all individual dimensional differences squared
  }
  sqrt(sum) #sqrt to find the final value for euclidean distance
}

In [33]:
df1 <- tibble(a = c(1,2,3,4,5),
              b = c(2,4,6,8,10),
              c = c(1,3,5,7,9)) # test dataframe 1
df2 <- tibble(a = c(2,3,4,5,6),
              b = c(3,5,7,9,10),
              c = c(4,0,5,3,8)) # test dataframe 2

df1 # allows you to see the table contents of df1
df2 # allows you to see the table contents of df2

a,b,c
<dbl>,<dbl>,<dbl>
1,2,1
2,4,3
3,6,5
4,8,7
5,10,9


a,b,c
<dbl>,<dbl>,<dbl>
2,3,4
3,5,0
4,7,5
5,9,3
6,10,8


In [45]:
distance(df1, df2) #tests whether the distance function as a 1:1 is correct

a
<dbl>
3.316625
3.316625
1.414214
4.242641
1.414214


In [59]:
# attempt to compute a distance between each row in df1 against every row of df2
val = 0 # initialize val
euclid_dist <- function(df1, df2){ #takes two dataframes
  for (i in 1:nrow(df1)){ # traverses all the row coordinates in df1
    for (j in 1:nrow(df2)){ # traverses each row coordinate of df2
      sample1 <- slice(df1, i) # picks up only the row of interest in df1 as a tibble
      sample2 <- slice(df2, j) # picks up only the row of interest in df2 as a tibble
      for (i in 1:3){ # for each of the the 3 variables x,y,z,
        val = val + (sample1[i]-sample2[i])^2 # take the slices of the rows and finds the distance
      }
    }
  }
  sqrt(val) # my logic seems to fall apart here because when i test this, i want a table of 25 rows,
} # where row 1 in df1 has a distance with row 1,2,3,4, and 5 of df2, and so forth. where did i go wrong?

In [41]:
slice(df1, 3) # demonstrates how a slice just takes your data from and row number and outputs a table

a,b,c
<dbl>,<dbl>,<dbl>
3,6,5


In [58]:
euclid_dist(df1, df2) # testing my incorrect function

next steps: once I get the bug out of my euclid_dist() function

- assign each calculation to the coordinates they computed the distance between
- group the distances by their "root", aka the point in df1 that each distance in df2 is computed from
- use arrange() and head(10) to find the smallest euclidean distances for each root value
- head() value is essentially the k of k-NN in this scenario
- find the average of those ten smallest distances from each df1 root point
- plot the value of the distance any point on a spherical radius of the root centroid against the root centroid coordinates
- assess pearson correlation.